## Context dependent models with CpG effect

CpG effect is deamination of 5-methy-cytosine at both positions, i.e. the C and the G (which is a C on the opposite strand). Thus the change is CpG->TpG OR CpG->CpA.

To complete the definition of the context dependent model, we also include the general nucleotide parameters.

### Relationship to the independent (nucleotide) process

If you omit the CpG parameters below and provide the dinucleotide state frequencies as the Kronecker product of nucleotide frequencies, then the log-likelihood of that model will be identical to the GN model with the same nucleotide frequencies.

> **Note**
> The branch length for the dinucleotide model is the expected number of substitutions per aligned *dinucleotide*. If memory serves correct, this will be exactly 1/2 that from the nucleotide model. So all genetic distances (the branch lengths in ENS) you are using need to be transformed into the expected number of substitutions per aligned *nucleotide*.

In [1]:
from itertools import permutations
import cogent3 as c3
from cogent3.evolve.predicate import MotifChange
from cogent3.evolve.ns_substitution_model import NonReversibleDinucleotide, NonReversibleCodon

def make_gn_preds():
    # making the model parameters (predictates) for
    # the General Nucleotide Markov model
    return [
        MotifChange(f, t, forward_only=True)
        for f, t in permutations("ACTG", 2)
        if f != "T" or t != "G"
    ]

def make_nr_cpg_preds_strand_asymetric():
    # strand specific CpG deamination rates, so separate
    # parameters for plus strand (CG->TG) and minus
    # strand (CG->CA)
    return [
        MotifChange("CG", "TG", forward_only=True),
        MotifChange("CG", "CA", forward_only=True),
    ]

def make_nr_cpg_preds_strand_symetric():
    # same CpG deamination rate on both strands
    # so one parameter
    return [
        # | is the binary or operator that combines the predicates
        # so we have the union of the two changes as a single parameter
        MotifChange("CG", "TG", forward_only=True) |
        MotifChange("CG", "CA", forward_only=True)
    ]

def _make_model(cls, **kwargs):
    return cls(**kwargs)

def GDN_CpG_ss(**kwargs):
    """return a dinucleotide model with strand symmetric CpG deamination"""
    ssym_preds = make_gn_preds() + make_nr_cpg_preds_strand_symetric()
    kwargs=dict(predicates=ssym_preds, optimise_motif_probs=True, name="GDN_CpG_ss")
    return _make_model(NonReversibleDinucleotide, **kwargs)

def GDN_CpG(**kwargs):
    """return a dinucleotide model with strand symmetric CpG deamination"""
    asym_preds = make_gn_preds() + make_nr_cpg_preds_strand_asymetric()
    kwargs=dict(predicates=asym_preds, optimise_motif_probs=True, name="GDN_CpG")
    return _make_model(NonReversibleDinucleotide, **kwargs)

In [2]:
aln = c3.get_dataset("primate-brca1")
tree = c3.get_dataset("primate-tree")

we drop gapped columns

In [3]:
aln = aln.take_seqs(["Chimpanzee", "Human", "Gorilla"])
aln1 = aln.omit_gap_pos(motif_length=3, allowed_gap_frac=0.0)
aln1

,0
Chimpanzee,TGTGGCACAAATACTCATGCCAGCTCATTACAGCATGAGAACAGCAGTTTATTACTCACT
Human,............................................................
Gorilla,............................................................


## Context dependent model's have a frame

This is obviously True for codon models in so much as an alignment is split into non-overlapping trinucleotides and those are modelled. But it's also true for dinucleotide in that the alignment is split into non-verlapping dinucleotides. We can check the robustness of the fit to *frame* by evaluating the other frame (slicing from 1). I should just point out that there is no known biological basis for the concept of a dinucleotide frame. It is purely something to consider with respect to data sampling and parameter estimation.

In [4]:
aln2 = aln[1:].omit_gap_pos(motif_length=2, allowed_gap_frac=0.0)
aln2

,0
Chimpanzee,GTGGCACAAATACTCATGCCAGCTCATTACAGCATGAGAACAGCAGTTTATTACTCACTA
Human,............................................................
Gorilla,............................................................


In [5]:
di_cpg = c3.get_app("model", GDN_CpG_ss(), time_het="max", show_progress=True, optimise_motif_probs=True)
di_cpg

model(sm=NonReversibleDinucleotide(name='GDN_CpG_ss'; params=['A>C', 'A>T',
'A>G', 'C>A', 'C>T', 'C>G', 'T>A', 'T>C', 'G>A', 'G>C', 'G>T', '(CG>TG |
CG>CA)']; num_motifs=16; motifs=['TT', 'TC', 'TA', 'TG', 'CT', 'CC', 'CA', 'CG',
'AT', 'AC', 'AA', 'AG', 'GT', 'GC', 'GA', 'GG'])), tree=None,
unique_trees=False, tree_func=None, name=None, optimise_motif_probs=True,
sm_args=None, lf_args=None, time_het='max', param_rules=None, opt_args=None,
lower=1e-06, upper=50, split_codons=False, show_progress=True, verbose=False)

In [6]:
result1 = di_cpg(aln1)
result1.lf

   0%|          |00:00<?

   0%|          |00:00<?

GDN_CpG_ss
log-likelihood = -3993.1753
number of free parameters = 54
===========================================================================
edge          parent    length    (CG>TG | CG>CA)      A>C      A>G     A>T
---------------------------------------------------------------------------
Chimpanzee    root        0.01               0.00     0.00    10.81    0.00
Human         root        0.01               7.44     0.00     1.96    0.00
Gorilla       root        0.00               0.00    14.14    42.55    0.00
---------------------------------------------------------------------------

continued: 
==================================================================
  C>A      C>G      C>T      G>A      G>C     G>T     T>A      T>C
------------------------------------------------------------------
 0.00    23.38    24.06    19.64    19.23    0.00    0.00    50.00
 1.38     2.78     2.39     5.05     1.14    0.00    1.00     2.02
30.18     0.00     0.00    50.00     0.00    0.00    0.00     0.00
------------------------------------------------------------------

============================================================================
  AA      AC      AG      AT      CA      CC      CG      CT      GA      GC
----------------------------------------------------------------------------
0.14    0.05    0.11    0.08    0.07    0.05    0.01    0.05    0.08    0.04
----------------------------------------------------------------------------

continued: 
============================================
  GG      GT      TA      TC      TG      TT
--------------------------------------------
0.04    0.05    0.07    0.04    0.06    0.06
--------------------------------------------

In [7]:
result2 = di_cpg(aln2)
result2.lf

   0%|          |00:00<?

   0%|          |00:00<?

GDN_CpG_ss
log-likelihood = -3971.4674
number of free parameters = 54
===========================================================================
edge          parent    length    (CG>TG | CG>CA)      A>C      A>G     A>T
---------------------------------------------------------------------------
Chimpanzee    root        0.01               0.00     0.00    10.87    0.00
Human         root        0.01              10.99     0.00     1.96    0.00
Gorilla       root        0.00               0.00    14.14    42.44    0.00
---------------------------------------------------------------------------

continued: 
==================================================================
  C>A      C>G      C>T      G>A      G>C     G>T     T>A      T>C
------------------------------------------------------------------
 0.00    23.49    24.05    19.83    19.36    0.00    0.00    50.00
 1.38     2.79     2.38     5.05     1.14    0.00    1.00     2.01
30.32     0.00     0.00    50.00     0.00    0.00    0.00     0.00
------------------------------------------------------------------

============================================================================
  AA      AC      AG      AT      CA      CC      CG      CT      GA      GC
----------------------------------------------------------------------------
0.15    0.05    0.10    0.07    0.08    0.03    0.01    0.06    0.08    0.04
----------------------------------------------------------------------------

continued: 
============================================
  GG      GT      TA      TC      TG      TT
--------------------------------------------
0.05    0.05    0.07    0.05    0.06    0.06
--------------------------------------------

TO DO: a codons variant of the above, change the base model class and add "omega" (see my earlier papers on this)